# input/output annotation 

In [1]:
#build the variable list and their comments
import re

def extract_plant_data_declarations(filename):
    """
    Extract data declarations from Fortran type blocks in the plant API data file.
    
    Returns a list of dictionaries containing variable name and comment for each declaration.
    """
    
    declarations = []
    
    with open(filename, 'r') as file:
        content = file.read()
    
    # Find all type blocks that start with 'type, public ::' and end with 'contains'
    type_pattern = r'type,\s*public\s*::\s*\w+.*?contains'
    type_blocks = re.findall(type_pattern, content, re.DOTALL | re.IGNORECASE)
    
    for block in type_blocks:
        # Extract individual declaration lines
        lines = block.split('\n')
        
        for line in lines:
            line = line.strip()
            
            # Skip empty lines, type declaration, and contains
            if not line or line.lower().startswith('type,') or line.lower().startswith('contains'):
                continue
            
            # Look for variable declarations (real, integer, character, logical)
            declaration_pattern = r'^\s*(real|integer|character|logical)\s*.*?::\s*([^!\s]+).*?(!.*)?$'
            match = re.match(declaration_pattern, line, re.IGNORECASE)
            
            if match:
                data_type = match.group(1)
                variable_part = match.group(2).strip()
                comment_part = match.group(3) if match.group(3) else ""
                
                # Extract variable name (before first parenthesis if present)
                var_name_match = re.match(r'([^(\s]+)', variable_part)
                if var_name_match:
                    variable_name = var_name_match.group(1)
                    
                    # Clean up comment (remove leading !)
                    comment = comment_part.lstrip('!').strip() if comment_part else ""
                    
                    declarations.append({
                        'variable_name': variable_name,
                        'comment': comment,
                        'full_declaration': line
                    })
    
    return declarations

def print_declarations(declarations):
    """Print the extracted declarations in a formatted way."""
    
    print("-" * 80)
    
    for i, decl in enumerate(declarations, 1):
        print(f"{i:2d}. Variable: {decl['variable_name']} !{decl['comment']}")

# Main execution

filename = '../../f90src/APIData/PlantAPIData.F90'

try:
    declarations = extract_plant_data_declarations(filename)
    print(f"Found {len(declarations)} data declarations:\n")    
#    print_declarations(declarations)
    
    # Also return the list for further processing if needed
#    print("List of extracted data:")
    decl_varname,decl_comment=[],[]
    for decl in declarations:
        decl_varname.append(decl['variable_name'])
        decl_comment.append(decl['comment'])
#        print(f"  {decl['variable_name']} - {decl['comment']}")
        
except FileNotFoundError:
    print(f"Error: File '{filename}' not found.")
except Exception as e:
    print(f"Error processing file: {e}")

filename = '../../f90src/Modelpars/GrosubPars.F90'

try:
    declarations = extract_plant_data_declarations(filename)
    print(f"Found {len(declarations)} data declarations:\n")    
#    print_declarations(declarations)
    
    # Also return the list for further processing if needed
#    print("List of extracted data:")
    for decl in declarations:
        decl_varname.append(decl['variable_name'])
        decl_comment.append(decl['comment'])
#        print(f"  {decl['variable_name']} - {decl['comment']}")
        
except FileNotFoundError:
    print(f"Error: File '{filename}' not found.")
except Exception as e:
    print(f"Error processing file: {e}")    

Found 639 data declarations:

Found 28 data declarations:



In [65]:
import re
import os

search_path = '../../f90src/Plant_bgc/'
    
files_to_process = ['ExtractsMod.F90','InitPlantMod.F90','InitVegBGC.F90','NoduleBGCMod.F90','NutUptakeMod.F90','PhotoSynsMod.F90','PlantBalMod.F90',
    'PlantBranchMod.F90','PlantDebugMod.F90','PlantDisturbByFireMod.F90','PlantDisturbByGrazingMod.F90','PlantDisturbByTillageMod.F90',
    'PlantDisturbsMod.F90','PlantNonstElmDynMod.F90','PlantPhenolMod.F90','RootGasMod.F90','RootMod.F90',
    'StomatesMod.F90','SurfaceRadiationMod.F90']

def separate_fortran_file(filename):
    """
    Separate a Fortran file into header, subroutines/functions, and tail sections.
    
    Returns a dictionary containing:
    - 'header': content from beginning to !header
    - 'subroutines': list of subroutines/functions
    - 'tail': content from !tail to end
    """
    
    with open(filename, 'r') as file:
        content = file.read()
    
    lines = content.split('\n')
    
    # Initialize sections
    header_lines = []
    subroutines = []
    tail_lines = []
    
    # State tracking
    in_header = True
    in_tail = False
    current_subroutine = []
    in_subroutine = False
    subroutine_name = ""
    
    i = 0
    while i < len(lines):
        line = lines[i]
        stripped_line = line.strip()
        
        # Check for header end marker
        if stripped_line.startswith('![header]') and in_header:
            header_lines.append(line)
            in_header = False
            i += 1
            continue
        
        # Check for tail start marker
        if stripped_line.startswith('![tail]'):
            in_tail = True
            tail_lines.append(line)
            i += 1
            continue
        
        # If we're in header section
        if in_header:
            header_lines.append(line)
        
        # If we're in tail section
        elif in_tail:
            tail_lines.append(line)
        
        # Otherwise, we're in the main body - look for subroutines/functions
        else:
            # Check for subroutine start
            subroutine_match = re.match(r'\s*subroutine\s+(\w+)', stripped_line, re.IGNORECASE)
            function_match = re.match(r'\s*function\s+(\w+)', stripped_line, re.IGNORECASE)
            
            if subroutine_match or function_match:
                # Save previous subroutine if exists
                if current_subroutine:
                    subroutines.append({
                        'name': subroutine_name,
                        'type': 'subroutine' if 'subroutine' in current_subroutine[0].lower() else 'function',
                        'content': '\n'.join(current_subroutine)
                    })
                
                # Start new subroutine/function
                current_subroutine = [line]
                subroutine_name = subroutine_match.group(1) if subroutine_match else function_match.group(1)
                in_subroutine = True
            
            elif in_subroutine:
                current_subroutine.append(line)
                
                # Check for subroutine/function end
                if (re.match(r'\s*end\s+subroutine', stripped_line, re.IGNORECASE) or 
                    re.match(r'\s*end\s+function', stripped_line, re.IGNORECASE)):
                    
                    # Save completed subroutine/function
                    subroutines.append({
                        'name': subroutine_name,
                        'type': 'subroutine' if 'subroutine' in current_subroutine[0].lower() else 'function',
                        'content': '\n'.join(current_subroutine)
                    })
                    
                    # Reset for next subroutine/function
                    current_subroutine = []
                    in_subroutine = False
                    subroutine_name = ""
            
            else:
                # Lines between subroutines (comments, blank lines, etc.)
                # These could be added to a separate section if needed
                pass
        
        i += 1
    
    # Handle case where file ends while in a subroutine (shouldn't happen in well-formed code)
    if current_subroutine and in_subroutine:
        subroutines.append({
            'name': subroutine_name,
            'type': 'subroutine' if 'subroutine' in current_subroutine[0].lower() else 'function',
            'content': '\n'.join(current_subroutine)
        })
    
    return {
        'header': '\n'.join(header_lines),
        'subroutines': subroutines,
        'tail': '\n'.join(tail_lines)
    }

def write_separated_files(filename, sections, output_dir=None):
    """
    Write the separated sections to individual files.
    """
    if output_dir is None:
        output_dir = os.path.dirname(filename) or '.'
    
    base_name = os.path.splitext(os.path.basename(filename))[0]
    
    # Write header
    header_file = os.path.join(output_dir, f"{base_name}_header.f90")
    with open(header_file, 'w') as f:
        f.write(sections['header'])
    print(f"Header written to: {header_file}")
    
    # Write each subroutine/function
    for i, sub in enumerate(sections['subroutines']):
        sub_file = os.path.join(output_dir, f"{base_name}_{sub['type']}_{sub['name']}.f90")
        print('sub_file=%s'%sub_file)
        with open(sub_file, 'w') as f:
            f.write(sub['content'])
        print(f"{sub['type'].capitalize()} '{sub['name']}' written to: {sub_file}")
    
    # Write tail
    if sections['tail'].strip():  # Only write if tail is not empty
        tail_file = os.path.join(output_dir, f"{base_name}_tail.f90")
        with open(tail_file, 'w') as f:
            f.write(sections['tail'])
        print(f"Tail written to: {tail_file}")

def print_summary(sections):
    """
    Print a summary of the separated sections.
    """
    print("\n" + "="*60)
    print("FILE SEPARATION SUMMARY")
    print("="*60)
    
    print(f"\nHEADER:")
    print(f"  Lines: {len(sections['header'].split(chr(10)))}")
    
    print(f"\nSUBROUTINES/FUNCTIONS: {len(sections['subroutines'])}")
    for i, sub in enumerate(sections['subroutines'], 1):
        lines = len(sub['content'].split('\n'))
        print(f"  {i:2d}. {sub['type'].upper()}: {sub['name']} ({lines} lines)")
    
    print(f"\nTAIL:")
    print(f"  Lines: {len(sections['tail'].split(chr(10)))}")


def separate_subroutine_function(content):
    """
    Separate a subroutine or function into three parts:
    1. From 'subroutine'/'function' to 'associate('
    2. From 'associate(' to line with only ')'
    3. Remaining content
    
    Returns a dictionary with the three parts or None if no associate block found.
    """
    
    lines = content.split('\n')
    
    # Find the associate statement
    associate_start = -1
    associate_end = -1
    
    for i, line in enumerate(lines):
        stripped = line.strip()
        
        # Look for associate statement (case insensitive)
        if re.match(r'\s*associate\s*\(', line, re.IGNORECASE):
            associate_start = i
            continue
        
        # Look for the closing parenthesis (only non-space character on the line)
        if associate_start != -1 and stripped == ')':
            associate_end = i
            break
    
    # If no associate block found, return the entire content as part 1
    if associate_start == -1:
        return {
            'has_associate': False,
            'part1': content,
            'part2': '',
            'part3': '',
            'associate_start': -1,
            'associate_end': -1
        }
    
    # Split into three parts
    part1_lines = lines[:associate_start]
    part2_lines = lines[associate_start:associate_end + 1]
    part3_lines = lines[associate_end + 1:]
    
    return {
        'has_associate': True,
        'part1': '\n'.join(part1_lines),
        'part2': '\n'.join(part2_lines),
        'part3': '\n'.join(part3_lines),
        'associate_start': associate_start,
        'associate_end': associate_end
    }

def extract_and_separate_subroutines_functions(content):
    """
    Extract all subroutines and functions from content and separate each into three parts.
    """
        
    lines = content.split('\n')
    
    subroutines_functions = []
    current_routine = []
    in_routine = False
    routine_name = ""
    routine_type = ""
    
    i = 0
    while i < len(lines):
        line = lines[i]
        stripped_line = line.strip()
        
        # Check for subroutine start
        subroutine_match = re.match(r'\s*subroutine\s+(\w+)', stripped_line, re.IGNORECASE)
        function_match = re.match(r'\s*function\s+(\w+)', stripped_line, re.IGNORECASE)
        
        if subroutine_match or function_match:
            # Save previous routine if exists
            if current_routine:
                routine_content = '\n'.join(current_routine)
                separated = separate_subroutine_function(routine_content)
                subroutines_functions.append({
                    'name': routine_name,
                    'type': routine_type,
                    'full_content': routine_content,
                    'separated': separated
                })
            
            # Start new routine
            current_routine = [line]
            routine_name = subroutine_match.group(1) if subroutine_match else function_match.group(1)
            routine_type = 'subroutine' if subroutine_match else 'function'
            in_routine = True
        
        elif in_routine:
            current_routine.append(line)
            
            # Check for routine end
            if (re.match(r'\s*end\s+subroutine', stripped_line, re.IGNORECASE) or 
                re.match(r'\s*end\s+function', stripped_line, re.IGNORECASE)):
                
                # Save completed routine
                routine_content = '\n'.join(current_routine)
                separated = separate_subroutine_function(routine_content)
                subroutines_functions.append({
                    'name': routine_name,
                    'type': routine_type,
                    'full_content': routine_content,
                    'separated': separated
                })
                
                # Reset for next routine
                current_routine = []
                in_routine = False
                routine_name = ""
                routine_type = ""
        
        i += 1
    
    # Handle case where file ends while in a routine
    if current_routine and in_routine:
        routine_content = '\n'.join(current_routine)
        separated = separate_subroutine_function(routine_content)
        subroutines_functions.append({
            'name': routine_name,
            'type': routine_type,
            'full_content': routine_content,
            'separated': separated
        })
    
    return subroutines_functions

def write_separated_parts(filename, routines, output_dir=None):
    """
    Write the separated parts to individual files.
    """
    if output_dir is None:
        output_dir = os.path.dirname(filename) or '.'
    
    base_name = os.path.splitext(os.path.basename(filename))[0]
    
    for routine in routines:
        name = routine['name']
        routine_type = routine['type']
        separated = routine['separated']
        
        # Create a subdirectory for each routine
        routine_dir = os.path.join(output_dir, f"{base_name}_{routine_type}_{name}")
        os.makedirs(routine_dir, exist_ok=True)
        
        # Write part 1 (declaration to associate)
        part1_file = os.path.join(routine_dir, f"{name}_part1_declaration.f90")
        with open(part1_file, 'w') as f:
            f.write(separated['part1'])
        
        if separated['has_associate']:
            # Write part 2 (associate block)
            part2_file = os.path.join(routine_dir, f"{name}_part2_associate.f90")
            with open(part2_file, 'w') as f:
                f.write(separated['part2'])
            
            # Write part 3 (remaining code)
            part3_file = os.path.join(routine_dir, f"{name}_part3_body.f90")
            with open(part3_file, 'w') as f:
                f.write(separated['part3'])
            
            print(f"{routine_type.capitalize()} '{name}' separated into 3 parts in: {routine_dir}")
        else:
            print(f"{routine_type.capitalize()} '{name}' has no associate block - only part 1 written to: {routine_dir}")

def print_separation_summary(routines):
    """
    Print a summary of the separation results.
    """
    print("\n" + "="*80)
    print("SUBROUTINE/FUNCTION SEPARATION SUMMARY")
    print("="*80)
    
    total_routines = len(routines)
    routines_with_associate = sum(1 for r in routines if r['separated']['has_associate'])
    
    print(f"\nTotal routines found: {total_routines}")
    print(f"Routines with associate blocks: {routines_with_associate}")
    print(f"Routines without associate blocks: {total_routines - routines_with_associate}")
    
    print(f"\nDetailed breakdown:")
    print("-" * 80)
    
    for i, routine in enumerate(routines, 1):
        name = routine['name']
        routine_type = routine['type']
        separated = routine['separated']
        
        total_lines = len(routine['full_content'].split('\n'))
        
        if separated['has_associate']:
            part1_lines = len(separated['part1'].split('\n'))
            part2_lines = len(separated['part2'].split('\n'))
            part3_lines = len(separated['part3'].split('\n'))
            
            print(f"{i:2d}. {routine_type.upper()}: {name} (Total: {total_lines} lines)")
            print(f"    Part 1 (Declaration): {part1_lines} lines")
            print(f"    Part 2 (Associate):   {part2_lines} lines")
            print(f"    Part 3 (Body):        {part3_lines} lines")
            print(f"    Associate block: lines {separated['associate_start']+1}-{separated['associate_end']+1}")
        else:
            print(f"{i:2d}. {routine_type.upper()}: {name} (Total: {total_lines} lines)")
            print(f"    No associate block found - entire content in Part 1")
        print()

def analyze_associate_blocks(routines):
    """
    Analyze the associate blocks found in the routines.
    """
    print("\n" + "="*80)
    print("ASSOCIATE BLOCK ANALYSIS")
    print("="*80)
    
    for routine in routines:
        if routine['separated']['has_associate']:
            name = routine['name']
            routine_type = routine['type']
            part2 = routine['separated']['part2']
            
            print(f"\n{routine_type.upper()}: {name}")
            print("-" * 40)
            print("Associate block content:")
            print(part2)
            print("-" * 40)

def extract_aliases_from_associate(part2_content):
    """
    Extract variable aliases from the associate block.
    Returns a dictionary mapping alias_var -> original_structure
    """
    aliases = {}
    
    if not part2_content.strip():
        return aliases
    
    lines = part2_content.split('\n')
    
    for line in lines:
        # Skip associate( and ) lines
        stripped = line.strip()
        if (stripped.lower().startswith('associate(') or 
            stripped == ')' or 
            stripped == '' or
            stripped.startswith('!')):
            continue
        
        cleaned_line = re.sub(r'[,&\s]*$', '', stripped)
        
        # Split by comma to handle multiple aliases on one line
        # But be careful with commas that might be inside comments
        alias_parts = []
        if '!' in cleaned_line:
            # Split at comment first, then handle the code part
            code_part = cleaned_line.split('!')[0].strip()
        else:
            code_part = cleaned_line
        
        # Remove trailing comma and ampersand from code part
        code_part = re.sub(r'[,&\s]*$', '', code_part)
        
        # Split by comma for multiple aliases
        parts = [part.strip() for part in code_part.split(',') if part.strip()]
        alias_parts.extend(parts)
        
        for alias_part in alias_parts:
            # Match pattern: alias => original, handling trailing comma/ampersand
            # Remove any trailing comma, ampersand, or whitespace
            clean_alias_part = re.sub(r'[,&\s]*$', '', alias_part.strip())
            
            alias_match = re.match(r'(\w+)\s*=>\s*(.+)', clean_alias_part)
            if alias_match:
                alias_var = alias_match.group(1).strip()
                original_structure = alias_match.group(2).strip()
                # Remove any trailing comma or ampersand from original_structure
                original_structure = re.sub(r'[,&\s]*$', '', original_structure)
                aliases[alias_var] = original_structure    
    return aliases

def analyze_variable_usage_in_part3(part3_content, aliases):
    """
    Analyze how each alias variable is used in part3.
    Returns a dictionary with usage classification for each alias.
    """
    usage_analysis = {}
    
    if not part3_content.strip():
        return usage_analysis
    
    # Initialize analysis for each alias
    for alias_var in aliases:
        usage_analysis[alias_var] = {
            'classification': 'unknown',
            'first_occurrence': None,
            'left_side_assignments': [],
            'right_side_usages': [],
            'function_calls': [],
            'other_usages': [],
            'all_occurrences': [],  # Track all occurrences in order
            'both_sides_same_line': [],  # Track lines where var appears on both sides
            'used_before_first_assignment': False
        }
    
    lines = part3_content.split('\n')
    
    # First pass: collect all occurrences in order
    for line_num, line in enumerate(lines, 1):
        # Skip comments and empty lines
        stripped = line.strip()
        if not stripped or stripped.startswith('!'):
            continue
        
        # Check each alias variable
        for alias_var in aliases:
            if alias_var in line:
                # Check if used in function call
                func_pattern = rf'\w+_func\s*\([^)]*{re.escape(alias_var)}[^)]*\)'
                if re.search(func_pattern, line, re.IGNORECASE):
                    occurrence = {
                        'line_num': line_num,
                        'line': line.strip(),
                        'type': 'function_call'
                    }
                    usage_analysis[alias_var]['function_calls'].append(occurrence)
                    usage_analysis[alias_var]['all_occurrences'].append(occurrence)
                    # Continue to check for other usages in the same line
                
                # Check for assignment statements
                # Look for patterns like: var = ... or var(...) = ...
                left_assign_pattern = rf'^\s*{re.escape(alias_var)}(\([^)]*\))?\s*='
                is_left_assignment = re.search(left_assign_pattern, stripped)
                
                # Check if variable appears on right side of assignment
                is_right_usage = False
                is_other_usage = False
                
                if '=' in line:
                    parts = line.split('=', 1)
                    if len(parts) == 2:
                        right_part = parts[1]
                        # Use word boundaries to avoid partial matches
                        if re.search(rf'\b{re.escape(alias_var)}\b', right_part):
                            is_right_usage = True
                
                # Check for both sides usage in same line
                if is_left_assignment and is_right_usage:
                    usage_analysis[alias_var]['both_sides_same_line'].append(line_num)
                
                # If not in assignment, it's other usage
                if not is_left_assignment and not is_right_usage and re.search(rf'\b{re.escape(alias_var)}\b', line):
                    # But skip if this usage is only in function call
                    if not re.search(func_pattern, line, re.IGNORECASE):
                        is_other_usage = True
                
                # Record occurrences (excluding function calls for classification purposes)
                if is_left_assignment:
                    occurrence = {
                        'line_num': line_num,
                        'line': line.strip(),
                        'type': 'left_assignment'
                    }
                    usage_analysis[alias_var]['left_side_assignments'].append(occurrence)
                    usage_analysis[alias_var]['all_occurrences'].append(occurrence)
                
                if is_right_usage:
                    occurrence = {
                        'line_num': line_num,
                        'line': line.strip(),
                        'type': 'right_usage'
                    }
                    usage_analysis[alias_var]['right_side_usages'].append(occurrence)
                    usage_analysis[alias_var]['all_occurrences'].append(occurrence)
                
                if is_other_usage:
                    occurrence = {
                        'line_num': line_num,
                        'line': line.strip(),
                        'type': 'other'
                    }
                    usage_analysis[alias_var]['other_usages'].append(occurrence)
                    usage_analysis[alias_var]['all_occurrences'].append(occurrence)
                
                # Set first occurrence if not set (excluding function calls)
                if (usage_analysis[alias_var]['first_occurrence'] is None and 
                    (is_left_assignment or is_right_usage or is_other_usage)):
                    if is_left_assignment:
                        usage_analysis[alias_var]['first_occurrence'] = {
                            'line_num': line_num,
                            'type': 'left_assignment'
                        }
                    elif is_right_usage:
                        usage_analysis[alias_var]['first_occurrence'] = {
                            'line_num': line_num,
                            'type': 'right_usage'
                        }
                    elif is_other_usage:
                        usage_analysis[alias_var]['first_occurrence'] = {
                            'line_num': line_num,
                            'type': 'other'
                        }
    
    # Second pass: analyze usage patterns and classify variables
    for alias_var in aliases:
        analysis = usage_analysis[alias_var]
        
        # Check if variable is not found in part3 (excluding function calls)
        total_usages = (len(analysis['left_side_assignments']) + 
                       len(analysis['right_side_usages']) + 
                       len(analysis['other_usages']))
        
        # If only function calls or no usage at all, classify as unknown
        if total_usages == 0:
            analysis['classification'] = 'unknown'
            continue
        
        # Sort all occurrences by line number to get chronological order (excluding function calls)
        all_occurrences = sorted([occ for occ in analysis['all_occurrences'] 
                                if occ['type'] != 'function_call'], key=lambda x: x['line_num'])
        
        # Check if variable has been used before its first left assignment
        first_left_assignment_line = None
        for occ in all_occurrences:
            if occ['type'] == 'left_assignment':
                first_left_assignment_line = occ['line_num']
                break
        
        if first_left_assignment_line:
            for occ in all_occurrences:
                if (occ['line_num'] < first_left_assignment_line and 
                    occ['type'] in ['right_usage', 'other']):
                    analysis['used_before_first_assignment'] = True
                    break
        
        # Classification logic:
        # INOPUT: Variable appears on both sides of = in same line OR 
        #         variable is found on left side of = but has been used before
        # OUTPUT: Variable is first found on left of = and doesn't meet inoput criteria
        # INPUT: Any other usage pattern
        # UNKNOWN: Not found in part3 OR only appears in function calls
        
        # Check for inoput conditions
        has_both_sides_same_line = len(analysis['both_sides_same_line']) > 0
        has_left_but_used_before = (len(analysis['left_side_assignments']) > 0 and 
                                   analysis['used_before_first_assignment'])
        
        if has_both_sides_same_line or has_left_but_used_before:
            analysis['classification'] = 'inoput'
        elif (all_occurrences and 
              all_occurrences[0]['type'] == 'left_assignment'):
            # First occurrence is left assignment and doesn't meet inoput criteria
            analysis['classification'] = 'output'
        else:
            # Any other usage pattern
            analysis['classification'] = 'input'
    
    return usage_analysis

# Main execution
            
    # Process both uploaded files

for filename in files_to_process:
    fullpath=search_path+filename
    if os.path.exists(fullpath):
        print(f"\nProcessing file: {filename}")
        print("-" * 40)
        
        try:
            sections = separate_fortran_file(fullpath)
            print_summary(sections)
            
            # Optionally write separated files
            #write_separated = input(f"\nWrite separated files for {filename}? (y/n): ").lower().strip()
            #if write_separated == 'y':
            #    write_separated_files(filename, sections)
            with open(filename, 'w') as f:
                f.write(sections['header'])
            
                for i, sub_funcs in enumerate(sections['subroutines']):
                    routine_sects=extract_and_separate_subroutines_functions(sub_funcs['content'])
                    f.write('\n!'+'-'*100+'\n')
                    f.write(routine_sects[0]['separated']['part1'])
                    f.write("\n")
                    aliases=extract_aliases_from_associate(routine_sects[0]['separated']['part2'])
                    if aliases:
                        f.write('  associate('+' '*54+'&\n')
                    a_len,t_len,num_alias=0,0,0
                    for alias in aliases:
                        a_len=max(a_len,len(alias.strip()))
                        t_len=max(t_len,len(aliases[alias].strip()))
                        num_alias=num_alias+1
                        
                    usage_analysis=analyze_variable_usage_in_part3(routine_sects[0]['separated']['part3'], aliases)
                    alias_v,target_v,label_v=[],[],[]
                    for alias in aliases:   
                        target=aliases[alias]
                        label=usage_analysis[alias]['classification']
                        alias_v.append(alias)
                        target_v.append(target)
                        label_v.append(label)
                    k=0    
                    for i in range(num_alias):    
                        if label_v[i] == 'input':
                            try:
                                index = decl_varname.index(alias_v[i])                             
                                if k==num_alias-1:
                                    f.write("    {:<{}} => {:<{}}   & !{:<{}}:{}\n".format(alias_v[i], a_len, target_v[i], t_len,label_v[i],7,re.sub(r'\s+', ' ',decl_comment[index])))
                                else:
                                    f.write("    {:<{}} => {:<{}}  ,& !{:<{}}:{}\n".format(alias_v[i], a_len, target_v[i], t_len,label_v[i],7,re.sub(r'\s+', ' ',decl_comment[index])))                                
                            except Exception as e:
                                if k==num_alias-1:
                                    f.write("    {:<{}} => {:<{}}   & !{:<{}}\n".format(alias_v[i], a_len, target_v[i], t_len,label_v[i],7))
                                else:
                                    f.write("    {:<{}} => {:<{}}  ,& !{:<{}}\n".format(alias_v[i], a_len, target_v[i], t_len,label_v[i],7))
                            k=k+1        
    
                    for i in range(num_alias):    
                        if label_v[i] == 'inoput':
                            try:
                                index = decl_varname.index(alias_v[i])                             
                                if k==num_alias-1:
                                    f.write("    {:<{}} => {:<{}}   & !{:<{}}:{}\n".format(alias_v[i], a_len, target_v[i], t_len,label_v[i],7,re.sub(r'\s+', ' ',decl_comment[index])))
                                else:
                                    f.write("    {:<{}} => {:<{}}  ,& !{:<{}}:{}\n".format(alias_v[i], a_len, target_v[i], t_len,label_v[i],7,re.sub(r'\s+', ' ',decl_comment[index])))
                            except Exception as e:
                                if k==num_alias-1:
                                    f.write("    {:<{}} => {:<{}}   & !{:<{}}\n".format(alias_v[i], a_len, target_v[i], t_len,label_v[i],7))
                                else:
                                    f.write("    {:<{}} => {:<{}}  ,& !{:<{}}\n".format(alias_v[i], a_len, target_v[i], t_len,label_v[i],7))
                            k=k+1            
    
    
                    for i in range(num_alias):    
                        if label_v[i] == 'output':
                            try:
                                index = decl_varname.index(alias_v[i])                             
                                if k==num_alias-1:
                                    f.write("    {:<{}} => {:<{}}   & !{:<{}}:{}\n".format(alias_v[i], a_len, target_v[i], t_len,label_v[i],7,re.sub(r'\s+', ' ',decl_comment[index])))
                                else:
                                    f.write("    {:<{}} => {:<{}}  ,& !{:<{}}:{}\n".format(alias_v[i], a_len, target_v[i], t_len,label_v[i],7,re.sub(r'\s+', ' ',decl_comment[index])))
                            except Exception as e:
                                if k==num_alias-1:
                                    f.write("    {:<{}} => {:<{}}   & !{:<{}}\n".format(alias_v[i], a_len, target_v[i], t_len,label_v[i],7))
                                else:
                                    f.write("    {:<{}} => {:<{}}  ,& !{:<{}}\n".format(alias_v[i], a_len, target_v[i], t_len,label_v[i],7))
                            k=k+1            
                    if aliases:
                        f.write('  )\n')
                    f.write(routine_sects[0]['separated']['part3'])
                    f.write('\n')
                f.write(sections['tail'])
        except Exception as e:
            print(f"Error processing {filename}: {e}")
    else:
        print(f"File not found: {filename}")


Processing file: ExtractsMod.F90
----------------------------------------

FILE SEPARATION SUMMARY

HEADER:
  Lines: 22

SUBROUTINES/FUNCTIONS: 5
   1. SUBROUTINE: extracts (28 lines)
   2. SUBROUTINE: TotalLitrFall (58 lines)
   3. SUBROUTINE: CalcTotalLeafArea (27 lines)
   4. SUBROUTINE: TotalGasandSoluteUptake (135 lines)
   5. SUBROUTINE: ExtractCanopyFluxes (117 lines)

TAIL:
  Lines: 3

Processing file: InitPlantMod.F90
----------------------------------------

FILE SEPARATION SUMMARY

HEADER:
  Lines: 22

SUBROUTINES/FUNCTIONS: 10
   1. SUBROUTINE: StartPlants (83 lines)
   2. SUBROUTINE: InitShootGrowth (50 lines)
   3. SUBROUTINE: PlantLitterFraction (234 lines)
   4. SUBROUTINE: PFTThermalAcclimation (45 lines)
   5. SUBROUTINE: InitDimensionsandUptake (117 lines)
   6. SUBROUTINE: InitPlantPhenoMorphoBio (207 lines)
   7. SUBROUTINE: InitMassBalance (71 lines)
   8. SUBROUTINE: InitPlantHeatWater (57 lines)
   9. SUBROUTINE: InitRootMychorMorphoBio (121 lines)
  10. SUBROU